#### **Library imports**

In [3]:
# Library imports
import pyforest
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from pprint import pprint
from time import sleep
import time

from turtle import forward
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.utils.data import DataLoader, TensorDataset

#### **Hyperparams and loading data**

In [78]:
train_edges = np.load('data/train_edges.npy')
# users = torch.LongTensor(train_edges[:, 0])
# items = torch.LongTensor(train_edges[:, 1])
# ratings = torch.FloatTensor(train_edges[:, 2])
user_list = train_edges[:, 0]
item_list = train_edges[:, 1]
rating_list = train_edges[:, 2].astype('float32')

n_users = 943 
n_items = 1682
n_samples = len(ratings)

#### **Defining collaborative filtering**

In [5]:
class CollaborativeFiltering(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(CollaborativeFiltering, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)

    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        dot = (u * i).sum(1)
        return torch.sigmoid(dot)

#### **Code for meta attack**

In [23]:
# start execution
start_time = time.time()

# some hyperparams
lr = 1
T = 10
Delta = 10
n_factors = 64

# store loss results in this list and later convert to dataframe 
results = []

# list of perturbations
perturbations = dict()
perturbations['edges'] = []
perturbations['metagrad'] = []

# print hyperparam config
print('-> Learning rate: ', lr)
print('-> T: ', T)
print('-> Delta: ', Delta)
print('-> Embedding size: ', n_factors)

for delta in range(Delta):
    # reload the users, items and ratings tensors
    users = torch.LongTensor(train_edges[:, 0])
    items = torch.LongTensor(train_edges[:, 1])
    ratings = torch.FloatTensor(train_edges[:, 2])

    # add those perturbations to "ratings"
    for index in perturbations['edges']:
        ratings[index] = 1

    # set requires_grad for ratings, to compute meta gradients
    ratings.requires_grad_()

    # makes code reproducible
    torch.manual_seed(0)

    # define model and loss
    model = CollaborativeFiltering(n_users, n_items, n_factors)
    p1, p2 = model.parameters()
    loss_fn = nn.BCELoss(reduction = 'mean')
    model.train()

    # inner loop training process
    for i in range(T):
        y_hat = model(users, items)
        loss = loss_fn(y_hat, ratings)
        results.append([delta, i, loss.item()])
        
        p1_grad = torch.autograd.grad(loss, p1, create_graph=True)
        p2_grad = torch.autograd.grad(loss, p2, create_graph=True)

        # compute inner parameter gradients
        with torch.no_grad():
            p1_new = p1 - lr * p1_grad[0]
            p2_new = p2 - lr * p2_grad[0]
            p1.copy_(p1_new)
            p2.copy_(p2_new)
    
    # compute meta gradient
    meta_grad = torch.autograd.grad(loss, ratings)[0]

    # select best edge to perturb
    max_meta_grad = -math.inf
    edge_to_add = -1
    for i in range(n_samples):
        if ratings[i] == 0: # search over only negative edges
            if meta_grad[i] > max_meta_grad:
                max_meta_grad = meta_grad[i]
                edge_to_add = i 

    perturbations['edges'].append(edge_to_add)
    perturbations['metagrad'].append(max_meta_grad.item())

# compute execution time
exec_time = int(time.time() - start_time)
exec_time = time.strftime("%Hh %Mm %Ss", time.gmtime(exec_time))
print('-> Execution time: {}'.format(exec_time))

# store results in CSV files
results = pd.DataFrame(results, columns = ['perturbs', 'iters', 'loss'])
results.to_csv('results/losses_Delta={}_T={}_LR={}_Factors={}'.format(Delta, T, lr, n_factors))

perturbations = pd.DataFrame(perturbations)
perturbations.to_csv('results/perturbations_Delta={}_T={}_LR={}_Factors={}'.format(Delta, T, lr, n_factors))

-> Learning rate:  1
-> T:  10
-> Delta:  10
-> Embedding size:  64
-> Execution time: 00h 00m 20s


#### **Checking stored results**

In [28]:
perturbations.head()

,edges,metagrad
0,141472,0.000193
1,152311,0.000175
2,173390,0.000174
3,5665,0.000164
4,112886,0.000164


In [29]:
results.head()

,perturbs,iters,loss
0,0,0,4.066596
1,0,1,4.065507
2,0,2,4.064427
3,0,3,4.062918
4,0,4,4.061419


In [1]:
torch.cuda.is_available()

<IPython.core.display.Javascript object>

True

#### **Code for meta attack in GPU**

In [84]:
# start execution
start_time = time.time()

# GPU settings (set use_gpu = -1 if you want to use CPU)
use_gpu = 0
if use_gpu == -1:
    device = 'cpu'
else:
    device = torch.device('cuda:{}'.format(str(use_gpu)) if torch.cuda.is_available() else 'cpu')

# some hyperparams
lr = 1
T = 10
Delta = 10
n_factors = 64

# store loss results in this list and later convert to dataframe 
results = []

# list of perturbations
perturbations = dict()
perturbations['edges'] = []
perturbations['metagrad'] = []

# print hyperparam config
print('-> Learning rate: ', lr)
print('-> T: ', T)
print('-> Delta: ', Delta)
print('-> Embedding size: ', n_factors)
print('-> Device: ', device)

# Load users and items data as tensors
users = torch.tensor(user_list, device = device)
items = torch.tensor(item_list, device = device)

for delta in range(Delta):
    # reload the ratings tensor
    ratings = torch.tensor(rating_list, device = device)

    # add those perturbations to "ratings" (think of a more efficient way to do this)
    for index in perturbations['edges']:
        ratings[index] = 1

    # set requires_grad for ratings, to compute meta gradients
    ratings.requires_grad_()

    # makes code reproducible
    torch.manual_seed(0)

    # define model and loss
    model = CollaborativeFiltering(n_users, n_items, n_factors)
    model.to(device)
    p1, p2 = model.parameters()
    loss_fn = nn.BCELoss(reduction = 'mean')
    model.train()

    # inner loop training process
    for i in range(T):
        y_hat = model(users, items)
        loss = loss_fn(y_hat, ratings)
        results.append([delta, i, loss.item()])
        
        p1_grad = torch.autograd.grad(loss, p1, create_graph=True)
        p2_grad = torch.autograd.grad(loss, p2, create_graph=True)

        # compute inner parameter gradients
        with torch.no_grad():
            p1_new = p1 - lr * p1_grad[0]
            p2_new = p2 - lr * p2_grad[0]
            p1.copy_(p1_new)
            p2.copy_(p2_new)
    
    # compute meta gradient
    meta_grad = torch.autograd.grad(loss, ratings)[0]

    # select best edge to perturb
    max_meta_grad = -math.inf
    edge_to_add = -1
    for i in range(n_samples):
        if ratings[i] == 0: # search over only negative edges
            if meta_grad[i] > max_meta_grad:
                max_meta_grad = meta_grad[i]
                edge_to_add = i 

    perturbations['edges'].append(edge_to_add)
    perturbations['metagrad'].append(max_meta_grad.item())

# compute execution time
exec_time = int(time.time() - start_time)
exec_time = time.strftime("%Hh %Mm %Ss", time.gmtime(exec_time))
print('-> Execution time: {}'.format(exec_time))

# store results in CSV files
results = pd.DataFrame(results, columns = ['perturbs', 'iters', 'loss'])
results.to_csv('results/losses_Delta={}_T={}_LR={}_Factors={}'.format(Delta, T, lr, n_factors))

perturbations = pd.DataFrame(perturbations)
perturbations.to_csv('results/perturbations_Delta={}_T={}_LR={}_Factors={}'.format(Delta, T, lr, n_factors))

-> Learning rate:  1
-> T:  10
-> Delta:  10
-> Embedding size:  64
-> Device:  cuda:0
-> Execution time: 00h 01m 22s


In [11]:
# check available devices
print('#GPUs: ', torch.cuda.device_count())

#GPUs:  8


#### **Some experiments**

In [85]:
use_gpu = 0
if use_gpu == -1:
    device = 'cpu'
else:
    device = torch.device('cuda:{}'.format(str(use_gpu)) if torch.cuda.is_available() else 'cpu')
print('current device: ', device)

model = CollaborativeFiltering(n_users, n_items, n_factors)
model.to(device)
print('model params in gpu: ', next(model.parameters()).is_cuda)

users = torch.tensor(user_list, device = device)
items = torch.tensor(item_list, device = device)
ratings = torch.tensor(rating_list, device = device)
print('users in gpu: ', users.is_cuda)
print('items in gpu: ', items.is_cuda)
print('ratings in gpu: ', ratings.is_cuda)

p1, p2 = model.parameters()
# print(type(p1))
print('p1 in gpu: ', p1.is_cuda)
# print(p2.is_cuda)

model.train()
y_hat = model(users, items)
print('y hat: ', y_hat)
print('y hat in gpu: ', y_hat.is_cuda)

print(ratings)
print(y_hat.dtype)
print(ratings.dtype)

loss = loss_fn(y_hat, ratings)
print('loss: ', loss)

p1_grad = torch.autograd.grad(loss, p1, create_graph=True)
# print('p1 grad: ', p1_grad)


current device:  cuda:0
model params in gpu:  True
users in gpu:  True
items in gpu:  True
ratings in gpu:  True
p1 in gpu:  True
y hat:  tensor([9.9797e-01, 1.4224e-04, 1.0000e+00,  ..., 9.8270e-03, 7.5457e-01,
        9.9830e-01], device='cuda:0', grad_fn=<SigmoidBackward0>)
y hat in gpu:  True
tensor([1., 1., 0.,  ..., 1., 1., 0.], device='cuda:0')
torch.float32
torch.float32
loss:  tensor(4.0545, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
